In [1]:
import os

go_up_n_directories = lambda path, n: os.path.abspath(
    os.path.join(*([os.path.dirname(path)] + [".."] * n))
)
os.chdir(go_up_n_directories(os.getcwd(), 4))  # run once (otherwise restart kernel)
print(os.getcwd())

/home/tim/Development/OCPPM


In [2]:
# Python natives
import lightgbm as lgb
import pprint
import numpy as np
from ocpa.algo.predictive_monitoring.obj import Feature_Storage
from ocpa.algo.predictive_monitoring import tabular
import pickle
import pandas as pd
import time
from datetime import datetime
import re
import sklearn.metrics as metrics
import utilities.evaluation_utils as evaluation_utils

In [14]:
feature_storage_file = (
    "data/OTC/feature_encodings/EFG/efg/raw/OTC_split_[C2_P2_P3_O3_eas].fs"
)
target = "event_remaining_time"

In [9]:
with open(feature_storage_file, "rb") as bin:
    feature_storage: Feature_Storage = pickle.load(bin)

In [ ]:
train_eft_in_file = "data/OTC/feature_encodings/baselines/EFT/eft_train.csv"
test_eft_in_file = "data/OTC/feature_encodings/baselines/EFT/eft_test.csv"

In [11]:
# flatten EFG with same train/test split
eft_train = tabular.construct_table(
    feature_storage, feature_storage.train_indices + feature_storage.validation_indices
)
eft_test = tabular.construct_table(feature_storage, feature_storage.test_indices)

# rename columns that contain JSON special characters (as they are not supported by LightGBM)
rename_columns = lambda col_name: re.sub("[^A-Za-z0-9_]+", "", str(col_name))
eft_train = eft_train.rename(columns=rename_columns)
eft_test = eft_test.rename(columns=rename_columns)

In [15]:
# import dataset into LightGBM framework
X_train, y_train = (
    eft_train.drop(columns=[target]),
    eft_train.loc[:, target],
)
X_valid, y_valid = (
    eft_test.drop(columns=[target]),
    eft_test.loc[:, target],
)

train_data = lgb.Dataset(
    X_train,
    label=y_train,
)
valid_data = lgb.Dataset(
    X_valid,
    label=y_valid,
)

In [6]:
params = {
    "objective": "regression",
    "metric": ["mse", "mae", "mape", "rmse"],
    "num_boost_round": 100,
    "stopping_rounds": 100,
}

start_train_time = datetime.now()
bst = lgb.train(
    params,
    train_data,
    valid_sets=[valid_data],
    callbacks=[lgb.early_stopping(params["stopping_rounds"])],
)
total_train_time = datetime.now() - start_train_time

/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.189991 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1881
[LightGBM] [Info] Number of data points in the train set: 402299, number of used features: 20
[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Info] Start training from score 0.005967
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[94]	valid_0's l2: 0.606287	valid_0's l1: 0.603978	valid_0's mape: 0.52503	valid_0's rmse: 0.778645


In [10]:
pred_start_time = time.time()
y_train_preds = bst.predict(X_train)
train_pred_time = time.time() - pred_start_time

pred_start_time = time.time()
y_valid_preds = bst.predict(X_valid)
valid_pred_time = time.time() - pred_start_time

In [12]:
eval_train = evaluation_utils.get_evaluation(
    y_train, y_train_preds, regression=True, time=train_pred_time
)
eval_train["report"]["training_time"] = total_train_time
eval_valid = evaluation_utils.get_evaluation(
    y_valid, y_valid_preds, regression=True, time=valid_pred_time
)

experiment_settings = {"experiment_settings": params}
evaluation_report = {"Train": eval_train, "Validation": eval_valid}

pprint.pprint(evaluation_report)

{'Train': {'report': {'MAE': 0.5498749211663642,
                      'MAPE': 2.8504579895842856,
                      'MSE': 0.5145465752313324,
                      'R^2': 0.47655206910375114,
                      'prediction_time': 0.655620813369751,
                      'training_time': datetime.timedelta(seconds=32, microseconds=156053)}},
 'Validation': {'report': {'MAE': 0.6039783080364023,
                           'MAPE': 2.820573074678381,
                           'MSE': 0.6062872931855923,
                           'R^2': 0.21476353684246918,
                           'prediction_time': 0.12030315399169922}}}
